In [1]:
import Graph_Amazon_Movies as gam
import Amazon_Movie_Parser as amp
import numpy as np
import pandas as pd
from networkx.algorithms import bipartite
import networkx as nx
import Amazon_Movie_Parser as prs
import datetime
from numpy import savetxt
import Computations as comp

## Preprocessing

In [2]:
grpComp = comp.GraphComp()

grp = gam.Graph_Amazon()
file_name='data/movies.txt'; n_movies=40000; prs_out='dictionary'

max_connected_gr_amazon_movies = grpComp.Create_Bipartite_Giant_Component(grp, file_name, n_movies, prs_out)

bottom_nodes, top_nodes = bipartite.sets(max_connected_gr_amazon_movies)

grpComp.health_check(grp, max_connected_gr_amazon_movies, bottom_nodes, top_nodes)

Calculation time-Crete graph from the movies file: 0:00:35.553900
max_connected_gr_amazon_movies:
Movies: 788 ... starts with B: 655
Users: 29294 ... starts with A: 29294
Is connected: True ... Is bipartite: True
BEFORE: # of nodes: 30082 ... # of edges: 38233


In [3]:
P_norm, R, R_zero = grpComp.Generate_P_Matrix(max_connected_gr_amazon_movies, bottom_nodes, top_nodes)

node_list, bottom_node_list, top_node_list, n_reviews, n_users, n_movies = \
grpComp.generate_node_lists(max_connected_gr_amazon_movies, bottom_nodes, top_nodes)

movie_indexes, user_indexes = grpComp.movie_user_indexes(node_list, top_node_list)

P to_numpy_matrix: 2020-05-13 14:37:42.526251
P normalize: 2020-05-13 14:37:47.523187
P finalize: 2020-05-13 14:37:57.418704
Calculation time-P matrix generation from the graph: 0:00:14.892449
Calculation time-user-movie indexes: 0:00:13.553713

15 movie indexes just to give some samples:
[1, 9, 23, 48, 51, 61, 71, 379, 397, 404, 415, 600, 602, 1132, 1138]


## Random Walk

In [4]:
# R_vector
#ref_user_idx=828 #A3R8PXSFGY9MC2
ref_user_idx=997
R_vector = grpComp.random_walk_vector(P_norm, R[ref_user_idx], R_zero[ref_user_idx], beta=0.15, n_steps=20)
# for the vector, n_steps=3, n_movies=40K ... calculation time is 0.66 seconds...

step-0 completed .. date: 2020-05-13 14:38:14.573561
step-1 completed .. date: 2020-05-13 14:38:14.940711
step-2 completed .. date: 2020-05-13 14:38:15.300513
step-3 completed .. date: 2020-05-13 14:38:15.657567
step-4 completed .. date: 2020-05-13 14:38:16.009983
step-5 completed .. date: 2020-05-13 14:38:16.370404
step-6 completed .. date: 2020-05-13 14:38:16.736362
step-7 completed .. date: 2020-05-13 14:38:17.096998
step-8 completed .. date: 2020-05-13 14:38:17.457076
step-9 completed .. date: 2020-05-13 14:38:17.818112
step-10 completed .. date: 2020-05-13 14:38:18.162386
step-11 completed .. date: 2020-05-13 14:38:18.517415
step-12 completed .. date: 2020-05-13 14:38:18.877436
step-13 completed .. date: 2020-05-13 14:38:19.231372
step-14 completed .. date: 2020-05-13 14:38:19.578819
step-15 completed .. date: 2020-05-13 14:38:19.925938
step-16 completed .. date: 2020-05-13 14:38:20.267880
step-17 completed .. date: 2020-05-13 14:38:20.631258
step-18 completed .. date: 2020-05-13 

## Summary

In [5]:
# R_vector
df_summary, movie_list_ref, movie_by_reference_user = \
grpComp.similarity_check_vector(R_vector, max_connected_gr_amazon_movies, movie_indexes, ref_user_idx, node_list, top_similarity=40)


R matrix values for the user (r vector of the user): 
[[[1.28611599e-04 3.39100724e-05 9.07897665e-04 ... 5.15032916e-06
   3.86274687e-06 6.43791146e-06]]]

 Top similarities: [30, 4356, 828, 2206, 79, 3109, 3144, 641, 780, 1238, 756, 2147, 5379, 5606, 815, 3119, 7265, 5548, 332, 1935, 3828, 3720, 7236, 1664, 3155, 2888, 3863, 7336, 1253, 346, 7320, 412, 3736, 489, 652, 1346, 3060, 1089, 171, 2]

 user_similarity_top_score: [0.0003, 0.0003, 0.0003, 0.0003, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0006, 0.0006, 0.0006, 0.0008, 0.0009]

 user_similarity_top_non_zero: [30, 4356, 828, 2206, 79, 3109, 3144, 641, 780, 1238, 756, 2147, 5379, 5606, 815, 3119, 7265, 5548, 332, 1935, 3828, 3720, 7236, 1664, 3155, 2888, 3863, 7336, 1253, 346, 7320, 412, 3736, 489, 652, 1346, 3060, 1089, 171

In [6]:
df_summary_ratio = \
grpComp.similarity_summary_ratio(df_summary, movie_list_ref, max_connected_gr_amazon_movies, movie_by_reference_user)

#node_check_f(usr_idx = 0)


Total # of MOVIEs: 788 USERS: 29294 REVIEWs: 38233,  in the graph


                     user_movie  ratio_similar  review_count
0  (A3LZBOBV9H1HDV, B0001G6PZC)           22.5           539
1  (A3LZBOBV9H1HDV, B004EPYZQ2)           30.0           467
2  (A3LZBOBV9H1HDV, B001OKUREO)           62.5           199
3  (A3LZBOBV9H1HDV, B000AMWIVM)           47.5           492
4  (A3LZBOBV9H1HDV, B0007OCG4W)           47.5           437
5  (A3LZBOBV9H1HDV, B00005JMZK)           40.0           503


In [7]:
#pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 300)
print(df_summary)

               user       movie  user_id  movie_id  is_reviewed score  \
0    A2582KMXLK2P06  B0001G6PZC       30       602        False   ...   
1    A2582KMXLK2P06  B004EPYZQ2       30      3613        False   ...   
2    A2582KMXLK2P06  B001OKUREO       30      7198         True   4.0   
3    A2582KMXLK2P06  B000AMWIVM       30      7846        False   ...   
4    A2582KMXLK2P06  B0007OCG4W       30     19550        False   ...   
5    A2582KMXLK2P06  B00005JMZK       30     21580        False   ...   
6    A12X2J7AVZ04IQ  B0001G6PZC     4356       602        False   ...   
7    A12X2J7AVZ04IQ  B004EPYZQ2     4356      3613        False   ...   
8    A12X2J7AVZ04IQ  B001OKUREO     4356      7198        False   ...   
9    A12X2J7AVZ04IQ  B000AMWIVM     4356      7846         True   5.0   
10   A12X2J7AVZ04IQ  B0007OCG4W     4356     19550         True   5.0   
11   A12X2J7AVZ04IQ  B00005JMZK     4356     21580        False   ...   
12   A3R8PXSFGY9MC2  B0001G6PZC      828       602 

## VALIDATION TESTS

In [8]:
grpComp_v = comp.GraphComp()
grp_v = gam.Graph_Amazon()
# start_index_v >> here index means record, in reality index=9*start_index
file_name_v='data/movies.txt'; start_index_v=40000; n_movies_v=10000; prs_out_v='dictionary'

max_connected_gr_amazon_movies_VAL = \
grpComp_v.Create_Bipartite_Giant_Component_VAL(grp_v, file_name_v, start_index_v, n_movies_v, prs_out_v)

bottom_nodes_v, top_nodes_v = bipartite.sets(max_connected_gr_amazon_movies_VAL)

grpComp_v.health_check(grp_v, max_connected_gr_amazon_movies_VAL, bottom_nodes_v, top_nodes_v)

# We will need P_VAL to compare the results between our prediction and the reality
# Well.... actually using edge check might be more convenient...
#P_norm_v, R_v, R_zero_v = Generate_P_Matrix(max_connected_gr_amazon_movies_VAL, bottom_nodes_v, top_nodes_v)

node_list_v, bottom_node_list_v, top_node_list_v, n_reviews_v, n_users_v, n_movies_v = \
grpComp_v.generate_node_lists(max_connected_gr_amazon_movies_VAL, bottom_nodes_v, top_nodes_v)

movie_indexes_v, user_indexes_v = grpComp_v.movie_user_indexes(node_list_v, top_node_list_v)



Create_Bipartite_VALIDATION is running...
Calculation time-Crete graph from the movies file: 0:00:01.963216
max_connected_gr_amazon_movies:
Movies: 176 ... starts with B: 150
Users: 7284 ... starts with A: 7284
Is connected: True ... Is bipartite: True
BEFORE: # of nodes: 7460 ... # of edges: 9424
Calculation time-user-movie indexes: 0:00:00.575763

15 movie indexes just to give some samples:
[1, 35, 409, 426, 473, 479, 506, 552, 892, 936, 1120, 1476, 1500, 1505, 1507]


In [9]:
# what we are trying is, we will see if a movie in the new set was reviewed with our top neighbors or not
# if the movie satisfies the condition we will check, then we will guess as 
# there is an edge between the user and the movie. 
# we will create the R vector for this user and see the accuracy of our prediction
# we will predict edge for each movie in the second batch for the ref-user

#movie_FOR_reference_user = []
movie_FOR_reference_user = top_node_list_v

'''
# we will only use the NEW movies in the NEW batch to make a prediction
for movie_v in top_node_list_v:
    if movie_v not in top_node_list:
        movie_FOR_reference_user.append(movie_v)
'''

'\n# we will only use the NEW movies in the NEW batch to make a prediction\nfor movie_v in top_node_list_v:\n    if movie_v not in top_node_list:\n        movie_FOR_reference_user.append(movie_v)\n'

In [10]:
# here we will use the training parameters to make our prediction
user_name_val = node_list[ref_user_idx]
print('User name checked: {}'.format(user_name_val))
#ref_user_idx_val=9149
#R_vector = grpComp_v.random_walk_vector(P_norm, R[ref_user_idx_val], R_zero[ref_user_idx_val], beta=0.15, n_steps=20)

#movie_FOR_reference_user = top_node_list_v

df_summary_v, movie_list_ref_v, movie_by_reference_user_v = \
grpComp_v.similarity_check_vector_VAL(R_vector, movie_indexes, ref_user_idx, node_list, node_list_v, movie_FOR_reference_user, \
                            max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL, top_similarity=40)

User name checked: A3LZBOBV9H1HDV
R matrix values for the user (r vector of the user): 
[[[1.28611599e-04 3.39100724e-05 9.07897665e-04 ... 5.15032916e-06
   3.86274687e-06 6.43791146e-06]]]

 Top similarities: [30, 4356, 828, 2206, 79, 3109, 3144, 641, 780, 1238, 756, 2147, 5379, 5606, 815, 3119, 7265, 5548, 332, 1935, 3828, 3720, 7236, 1664, 3155, 2888, 3863, 7336, 1253, 346, 7320, 412, 3736, 489, 652, 1346, 3060, 1089, 171, 2]

 user_similarity_top_score: [0.0003, 0.0003, 0.0003, 0.0003, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0004, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0006, 0.0006, 0.0006, 0.0008, 0.0009]

 user_similarity_top_non_zero: [30, 4356, 828, 2206, 79, 3109, 3144, 641, 780, 1238, 756, 2147, 5379, 5606, 815, 3119, 7265, 5548, 332, 1935, 3828, 3720, 7236, 1664, 3155, 2888, 3863, 7336, 1253, 346, 7320, 412, 373

In [11]:
# we will check the values in max_connected_gr_amazon_movies not VAL because our reference is the first one
df_summary_ratio_v = \
grpComp_v.similarity_summary_ratio_VAL(df_summary_v, movie_list_ref_v, \
                             max_connected_gr_amazon_movies, max_connected_gr_amazon_movies_VAL)

Total # of MOVIEs: 176 USERS: 7284 REVIEWs: 9424,  in the graph


          movie  ratio_similar  review_count
0    B000A2SA64            0.0            34
1    B000H5U5I0            0.0            12
2    6302859433            0.0             1
3    6305290814            0.0             1
4    B0001DHSCY            0.0            21
5    B000068TWC            7.5            67
6    B00005Y6YP            0.0            18
7    6304578466            2.5            14
8    630356254X            0.0            31
9    B00005Y6Y3            0.0            13
10   0784011532            7.5            56
11   B004M7N40U            0.0            47
12   B0083SI986           35.0           529
13   B000N2HD7I            0.0            11
14   6303149405            0.0            26
15   B000065U31            2.5            10
16   0767805534            7.5           119
17   B002SL4LQ8            0.0             4
18   B00005Y6YC            0.0             8
19   B00005QG2N            0.0    

In [12]:
print('User: {} .. \nedges: {}'.format(user_name_val, max_connected_gr_amazon_movies_VAL.edges(user_name_val)))

User: A3LZBOBV9H1HDV .. 
edges: [('A3LZBOBV9H1HDV', 'B000XJSL9U')]


In [13]:
list(max_connected_gr_amazon_movies_VAL.edges(user_name_val))[0][1]

'B000XJSL9U'

In [14]:
lst_new_movies_reviewed = list(max_connected_gr_amazon_movies_VAL.edges(user_name_val))
for i in range(len(lst_new_movies_reviewed)):
    new_mov = lst_new_movies_reviewed[i][1]
    new_mov_summary = df_summary_ratio_v.query("movie=='" + new_mov + "'").values.tolist()
    print(new_mov_summary)
    
for i in range(len(lst_new_movies_reviewed)):
    new_mov = lst_new_movies_reviewed[i][1]
    new_mov_summary = df_summary_ratio.query("user_movie=='" + new_mov + "'").values.tolist()
    print(new_mov_summary)
    
#true_values=len(df_summary.query("movie=='B003AI2VGA' and is_reviewed==True"))

[['B000XJSL9U', 35.0, 434]]
[]


In [15]:
grpComp_v.movie_user_compare_datasets(top_nodes_v, top_nodes, bottom_nodes_v, bottom_nodes)

MOVIES >> YES: Exist in both .. NO: Exist in dataset-2 but not exist in dataset-1
count_YES: 1 .. count_NO: 175
USERS >> YES: Exist in both .. NO: Exist in dataset-2 but not exist in dataset-1
count_YES: 1894 .. count_NO: 5390
MOVIES >> YES: Exist in both .. NO: Exist in dataset-1 but not exist in dataset-2
count_YES: 1 .. count_NO: 787
USERS >> YES: Exist in both .. NO: Exist in dataset-1 but not exist in dataset-2
count_YES: 1894 .. count_NO: 27400


In [16]:
max_connected_gr_amazon_movies.get_edge_data('A1I7QGUDP043DG', 'B000ZLFALI')

In [17]:
max_connected_gr_amazon_movies_VAL.get_edge_data('A1I7QGUDP043DG', 'B00005LL26')

In [18]:
max_connected_gr_amazon_movies_VAL.edges('A1I7QGUDP043DG')

EdgeDataView([])

In [19]:
max_connected_gr_amazon_movies.edges('A1I7QGUDP04')

EdgeDataView([])

In [20]:
len(max_connected_gr_amazon_movies.edges('A1I7QGUDP04'))

0

In [21]:
'''
for b in bottom_node_list:
    b_edge = max_connected_gr_amazon_movies_VAL.edges(b)
    if len(b_edge) != 0:
        print('id={} .. user={} ... edge={}'.format(node_list.index(b), b, b_edge))
'''

"\nfor b in bottom_node_list:\n    b_edge = max_connected_gr_amazon_movies_VAL.edges(b)\n    if len(b_edge) != 0:\n        print('id={} .. user={} ... edge={}'.format(node_list.index(b), b, b_edge))\n"

In [22]:
#max_connected_gr_amazon_movies_VAL.edges()